In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")

In [30]:
def text_preprocess(text):
    """Use spaCy to remove stopwords and lemmatize
    """
    return " ".join(token.lemma_ for token in nlp(text)
                    if not token.is_stop)        

In [14]:
file = '../data/cleaned_text.csv'

In [29]:
df = pd.read_csv(file)

/Users/sueliu/Mudano/customer-complaints/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df['text'] = df['cleaned_text'].apply(text_preprocess)